In [ ]:
import matplotlib.patches as patches

import numpy as np 
import pandas as pd
from glob import glob

import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import gc
import os
from PIL import Image
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
KAGGLE=False
PATH="/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/" if KAGGLE else "./"

In [ ]:
train=pd.read_csv(PATH+'train.csv')
sub=pd.read_csv(PATH+'sample_submission.csv')

In [ ]:
train[train['class_id'] != 14].isnull().sum()

In [ ]:
train.class_name.unique()

> The dataset comprises 18,000 postero-anterior (PA) CXR scans in DICOM format, which were de-identified to protect patient privacy. All images were labeled by a panel of experienced radiologists for the presence of 14 critical radiographic findings as listed below:

0 - Aortic enlargement

1 - Atelectasis

2 - Calcification

3 - Cardiomegaly

4 - Consolidation

5 - ILD

6 - Infiltration

7 - Lung Opacity

8 - Nodule/Mass

9 - Other lesion

10 - Pleural effusion

11 - Pleural thickening

12 - Pneumothorax

13 - Pulmonary fibrosis

14 - No finding

**Columns**

**image_id** - unique image identifier

**class_name** - the name of the class of detected object (or "No finding")


**class_id** - the ID of the class of detected object

**rad_id** - the ID of the radiologist that made the observation

**x_min** - minimum X coordinate of the object's bounding box

**y_min** - minimum Y coordinate of the object's bounding box

**x_max** - maximum X coordinate of the object's bounding box

**y_max** - maximum Y coordinate of the object's bounding box

In [ ]:
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    if voi_lut:
        #apply voi lookup table if possible
        data = apply_voi_lut(dicom.pixel_array, dicom)
        
    #fix monochrome issue
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data.astype(np.float)
    
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    #modified pixel range on a greyscale (0-255)    
    return data

# Examples of diseases detection

In [ ]:
def get_rect_patch(x_min,y_min,x_max,y_max):
    width=x_max-x_min
    height=y_max-y_min
    rect=patches.Rectangle((x_min,y_min),width,height,ec='r', fc='none', lw=2.)
    return rect

In [ ]:
for classname in train.class_name.unique():
    fig,axs=plt.subplots(1,5,figsize=(20,15))
    fig.subplots_adjust(hspace = .1, wspace=.1)

    axs = axs.ravel()
    
    #sampling 3 images with corresponding disease
    samples=train[train['class_name'] == classname].sample(n=5,random_state=353)
    for i in range(5):
        axs[i].imshow(read_xray(PATH+"images/train/"+samples.iloc[i]['image_id']+".dicom"),cmap='gray')

        axs[i].set_title(str(classname))
        axs[i].set_xticklabels([])
        axs[i].set_yticklabels([])
        
        if classname != "No finding":
            x_min,y_min,x_max,y_max=samples.iloc[i]['x_min'],samples.iloc[i]['y_min'],samples.iloc[i]['x_max'],samples.iloc[i]['y_max']
            rect=get_rect_patch(x_min,y_min,x_max,y_max)
            
            axs[i].add_patch(rect)
plt.show()

**YOLO Format** is as follows:
* One row per object
* Each row is class x_center y_center width height format.
* Box coordinates must be in normalized xywh format (from 0 - 1). If your boxes are in pixels, divide x_center and width by image width, and y_center and height by image height.
* Class numbers are zero-indexed (start from 0).

In [ ]:
#Normalizing Data
def convert(size, box):
    #xmin,xmax,ymin,ymax
    dw = 1./(size[0])
    dh = 1./(size[1])
    x = (box[0] + box[1])/2.0 - 1
    y = (box[2] + box[3])/2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x*dw
    w = w*dw
    y = y*dh
    h = h*dh
    return (x,y,w,h)

In [ ]:
def convert_to_yolo(df,IsTrain=True):
    folder="train/" if IsTrain else "test/"
    global PATH
    
    r=pd.DataFrame(index=range(df.shape[0]),columns=['image_id','class','x','y','w','h','img_width','img_height'])
    for i,img_id in enumerate()
        print(str(i)+"/"+str(df.shape[0])+"\n")
        for idx,row in df.iterrows():
            box=[row['x_min'],row['x_max'],row['y_min'],row['y_max']]
            size=pydicom.read_file(PATH+"images/train/"+row['image_id']+".dicom").pixel_array.shape[::-1] 
            x,y,w,h=convert(size,box)
            r.iloc[idx]['img_width'],r.iloc[idx]['img_height'],r.iloc[idx]['image_id'],r.iloc[idx]['class'],r.iloc[idx]['x'],r.iloc[idx]['y'],r.iloc[idx]['w'],r.iloc[idx]['h'] = size[0],size[1],row['image_id'],row['class_id'],x,y,w,h
            gc.collect()
            i+=1
    return r

In [ ]:
r=convert_to_yolo(train)
r.to_csv('chest_xray.csv')

In [ ]:
r['image_id'].nunique()

In [ ]:
dicom_imgs_path=PATH+"train/"

train_imgs_path=PATH+"images/train/"
train_labels_path=PATH+"labels/train/"

test_imgs_path=PATH+"images/test/"

In [ ]:
n_classes=14

In [ ]:
#Creating .txt files in yolov5 format
for i in r.image_id.unique():
    img=read_xray(PATH+train_imgs_path+i+".dicom")
    cv2.imwrite("./" + train_imgs_path + i + ".jpg" , img)
    with open(PATH+train_labels_path+i+".txt","a") as f:
        f.seek(0)
        img_df=r[r["image_id"]== i]
        found=False
        nf_idxs=[]
        for idx,row in img_df.iterrows():
            if int(row['class']) == 14:
                nf_idxs += idx
                continue
            else:
                found=True
                f.write(str(row['class']) + " " + str(row['x']) + " " + str(row["y"]) + " " + str(row['w']) + " " + str(row['h']) + "\n")
        if found:
            for idx in nf_idxs:
                cv2.imwrite( "./" + train_imgs_path + i + "_NF_" + str(idx) + ".jpg" , img)
                f = open( PATH + train_labels_path + i + "_NF_" + str(idx) + ".txt" , "x" )
        f.close()

In [ ]:
#Converting test images into JPG
for i in sub.image_id.unique():
    img=read_xray(PATH+test_imgs_path+i+".dicom")
    cv2.imwrite("./" + test_imgs_path + i + ".jpg" , img)

In [ ]:
test=sub.copy()
test['width']=test['height']=0
for idx,row in test.iterrows():
    img = Image.open(PATH+test_imgs_path+row['image_id']+".jpg")
    test.loc[idx,'width'],test.loc[idx,'height']=img.size[0],img.size[1]

At this point i trained the model on a gpu cloud computing service ,namely : https://lambdalabs.com/.
For faster training i used : 
> python train.py torch.distributed.launch --nproc_per_node 4 ***etc...***

note that this notebook has to be adapted to the working dir to be able to run,as i didn't prepare it for kaggle deployement

In [ ]:
os.chdir(f"{PATH}yolov5-master")
!python train.py --weights "./weights/yolov5x.pt" --data ../chest_xray.yaml --epochs 60

In [ ]:
!python detect.py --weights "./runs/train/exp4/weights/best.pt" --save-txt --save-conf --sources "../images/test/" --img-size 640

In [ ]:
def yolo2voc(image_height, image_width, bboxes):
    """
    yolo => [xmid, ymid, w, h] (normalized)
    voc  => [x1, y1, x2, y1]
    
    """ 
    bboxes = bboxes.copy().astype(float) # otherwise all value will be 0 as voc_pascal dtype is np.int
    
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    
    return bboxes

In [ ]:
image_ids = []
PredictionStrings = []

for file_path in glob('runs/detect/exp/labels/*txt'):
    image_id = file_path.split('/')[-1].split('.')[0].split("\\")[1]
    w, h = int(test.loc[test.image_id==image_id]['width']),int(test.loc[test.image_id==image_id]['height'])
    f = open(file_path, 'r')
    data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
    data = data[:, [0, 5, 1, 2, 3, 4]]
    bboxes = list(np.round(np.concatenate((data[:, :2], np.round(yolo2voc(h, w, data[:, 2:]))), axis =1).reshape(-1), 1).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

In [ ]:
test_df=test.copy().drop("PredictionString",axis=1,inplace=False)
pred_df = pd.DataFrame({'image_id':image_ids,
                        'PredictionString':PredictionStrings})
sub_df = pd.merge(test_df, pred_df, on = 'image_id', how = 'left').fillna("14 1 0 0 1 1")
print(sub_df)
sub_df = sub_df.loc[:,['image_id', 'PredictionString']]
sub_df.to_csv('submission.csv',index = False)
sub_df.tail()